****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
# importing libraries
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
#reading data in pandas DataFrame
json_data = pd.read_json('world_bank_projects.json')

### 1. Find the 10 countries with most projects 
I am using three different columns ('countryname', 'countrycode', 'countryshortname') to compare resulsts. Method .value_counts() counts unique values and returns them in descending order

In [3]:
#using full country name
json_data.countryname.value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

In [4]:
#using country code
json_data.countrycode.value_counts().head(10)

ID    19
CN    19
VN    17
IN    16
RY    13
NP    12
BD    12
MA    12
MZ    11
3A    11
Name: countrycode, dtype: int64

In [5]:
#using short name of country
json_data.countryshortname.value_counts().head(10)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Morocco               12
Nepal                 12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64

### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
First will reload data as a string and using normalize function will create the table of project codes and names

In [6]:
data = json.load(open('world_bank_projects.json'))
all_codes = json_normalize(data, 'mjtheme_namecode')

#investigating unique values
unique_values = all_codes.drop_duplicates()
unique_values.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


It looks like some of the project themes are not listed. However, each of them has assigned code. Therefore I will count them.

In [7]:
#first, creating list of names and corresponding codes, codes will be used as index
unique_codes = unique_values[unique_values.name != ''].set_index('code')

#counting project codes occurance
top_10_codes = all_codes.code.value_counts().head(10)

#merging two tables on indexes and sorting in descending order
top_10_themes = pd.merge(unique_codes, top_10_codes, left_index=True, right_index=True).sort_values('code', ascending=False)

#setting meaningful column names
top_10_themes.columns = ['project_theme', 'count']
top_10_themes.index.name = 'theme_code'

#displaying results
top_10_themes

,project_theme,count
theme_code,,
11,Environment and natural resources management,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50


Now I will visualize results

In [8]:
top_10_themes.plot(kind='bar', x='project_theme', y='count', title='10 Major Project Themese', legend=False)

### 3.In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
Will start by changing unique_codes data frame to a disctionary

In [9]:
unique_dict = unique_codes.to_dict()

#only inside dictionary is needed
unique_dict = unique_dict['name']

Now with help of this dictionary I will add missing theme values in the column themes that was extracted in a previous task

In [10]:
#extracting column in question from data frame
themes = json_data['mjtheme_namecode']

#looping over themes and inside the loop, iterating over each list (each element in themes is a list of dictionaries) and filling missing vallues
for line in themes:
    for item in line:
        if item['name'] == '':
            item['name'] = unique_dict[item['code']]
            

Quick check to ensure all data is filled

In [11]:
clear_codes = pd.DataFrame()
for line in themes:
    codes = json_normalize(line)
    clear_codes = pd.concat([clear_codes, codes])
clear_codes.drop_duplicates()

,code,name
0,8,Human development
1,11,Environment and natural resources management
0,1,Economic management
1,6,Social protection and risk management
0,5,Trade and integration
1,2,Public sector governance
0,7,Social dev/gender/inclusion
1,4,Financial and private sector development
0,10,Rural development
0,9,Urban development


Above table shows that there are no missing values left. Now I will create a new data frame  by copying original json_data one and swapping the mjtheme_namecode for the themes

In [12]:
filled_data = json_data
filled_data['mjtheme_namecode'] = themes